In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random

def multinorm_translate(_mu,_sigma):
    dim = _sigma.shape[0]
    #compute _sigma^(-1/2)
    eigva, eigvc = np.linalg.eig(_sigma)
    eigvc = np.matrix(eigvc)
    eigva_half = np.diag(eigva**0.5)
    sigma_half = eigvc.T * eigva_half * eigvc

    #normalized gaussion
    point = [random.gauss(0,1) for i in range(dim)]
    #transform
    point = point * sigma_half + _mu

    return np.array(point)[0].tolist()

mu = np.array([5,5])
sigma = np.array([[4,0],[0,3]])
PointNum = 300
features = []
for i in range(PointNum):
    point = multinorm_translate(mu,sigma)
    features.append(point)

X = []
Y = []
for item in features:
    X.append(item[0])
    Y.append(item[1])
    
plt.scatter(X, Y , 30, color ='r', marker = 'o')
plt.show()



In [32]:
import matplotlib.pyplot as plt
import numpy as np
import random

def multinorm_translate(_mu,_sigma):
    dim = _sigma.shape[0]
    #compute _sigma^(-1/2)
    eigva, eigvc = np.linalg.eig(_sigma)
    eigvc = np.matrix(eigvc)
    eigva_half = np.diag(eigva**0.5)
    sigma_half = eigvc.T * eigva_half * eigvc

    #normalized gaussion
    point = [random.gauss(0,1) for i in range(dim)]
    #transform
    point = point * sigma_half + _mu

    return np.array(point)[0].tolist()

def gauss_pdf(point, mu, sigma):
    d = point.shape[0]
    p = 1.0/((2*np.pi)**(d*0.5) * np.linalg.det(sigma)**(0.5)) * np.exp(-0.5 * (np.dot(np.dot((point-mu), np.linalg.inv(sigma)), (point-mu).T)))
    return p

def Expect(data, mu, sigma, w):
    reslist=[]
    for i in range(len(data)):
        plist = []
        for j in range(len(w)):
            p = w[j] * gauss_pdf(data[i],mu[j],sigma[j]);
            plist.append(p.tolist()[0][0])
        psum = sum(plist)
        plist = [p/psum for p in plist]
        reslist.append(plist)
    return reslist
        
def Maximize(pl, data, mu):
    d = data.shape[1]
    new_w = []
    new_mu = []
    new_sigma =[]
    for i in range(len(mu)):
        p = 0
        _mu = np.zeros([1,dim])
        _sigma =  np.zeros([dim,dim])
        for j in range(len(data)):
            p += pl[j][i]
            _mu += np.array(pl[j][i]) * data[j]
            _x = np.matrix(data[j]-mu[i])
            _sigma += np.array(_x.T * _x) * pl[j][i]

        new_w.append(p/len(data))
        new_mu.append(_mu/p)
        new_sigma.append(_sigma/p)
    
    return new_w,new_mu,new_sigma


K = 2

mu1 = np.array([5,5])
sigma1 = np.array([[4,0],[0,4]])
PointNum1 = 500

features = []
for i in range(PointNum1):
    point = multinorm_translate(mu1,sigma1)
    features.append(point)

mu2 = np.array([-3,-2])
sigma2 = np.array([[2,0],[0,5]])
PointNum2 = 500

for i in range(PointNum2):
    point = multinorm_translate(mu2,sigma2)
    features.append(point)

features = np.matrix(features)
mean = sum(features) / len(features)

print mean
dim = sigma1.shape[0]
sigma0 = np.zeros([dim,dim])
for i in range(features.shape[0]):
    tem_x = features[i,:] - mean
    sigma0 += np.array(tem_x.T * tem_x)
sigma0 /= 1.0 * features.shape[0]

Weight = [1.0/K for i in range(K)]
Mu = [mean + random.uniform(-6, 6) for i in range(K)]
Sigma = [sigma0 for i in range(K)]

print Mu
print Sigma


iters = 0
delta = 100
while(delta > 0.0001):
    iters += 1
    
    w_old = Weight
    mu_old = Mu
    sigma_old = Sigma
    Plist = Expect(features, Mu, Sigma, Weight)
    [Weight, Mu, Sigma] = Maximize(Plist, features, Mu)
    
    delta = sum(sum(sum(abs(np.array(Mu)- np.array(mu_old)))))
    print "iteration %d, delta = %f" % (iters, delta)

print "Weight of %d components: " % (K), Weight
print "Mu of %d components: " % (K), Mu
print "Sigma of %d components: " % (K), Sigma

[[ 0.99721545  1.55819015]]
[matrix([[ 4.40675988,  4.96773457]]), matrix([[-2.02758145, -1.46660676]])]
[array([[ 18.82572511,  13.79014242],
       [ 13.79014242,  16.39057851]]), array([[ 18.82572511,  13.79014242],
       [ 13.79014242,  16.39057851]])]
iteration 1, delta = 2.881408
iteration 2, delta = 1.270437
iteration 3, delta = 1.770607
iteration 4, delta = 1.455090
iteration 5, delta = 0.430192
iteration 6, delta = 0.025113
iteration 7, delta = 0.001911
iteration 8, delta = 0.000483
iteration 9, delta = 0.000135
iteration 10, delta = 0.000037
Weight of 2 components:  [0.49946313850131613, 0.5005368614986839]
Mu of 2 components:  [array([[ 4.98288431,  5.0504137 ]]), array([[-2.97990358, -1.92654208]])]
Sigma of 2 components:  [array([[ 4.13112066, -0.10118119],
       [-0.10118119,  3.57904376]]), array([[ 1.81985231, -0.09651643],
       [-0.09651643,  4.86180803]])]
